<a href="https://colab.research.google.com/github/abhishekshah5486/aganitha_assignment/blob/main/aganitha_assignment_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import requests
import re
import csv
import xml.etree.ElementTree as ET
from typing import List, Dict, Tuple

In [15]:
BASE_URL = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
EFETCH_URL = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"

In [16]:
class PubMedFetcher():
  def __init__(self, search_query: str, batch_size: int = 200):
    self.search_query = search_query
    self.batch_size = batch_size
    self.pubmed_ids = None

  def fetch_pubmed_ids(self) -> List[str]:
    params = {
        'db': 'pubmed',
        'term': self.search_query,
        'usehistory': 'y'
    }
    pubmed_ids = []
    try:
      response = requests.get(BASE_URL, params=params, timeout=10)
      response.raise_for_status()
      root = ET.fromstring(response.text)
      self.pubmed_ids = [id_elem.text for id_elem in root.findall('.//Id')]
      if not self.pubmed_ids:
        print("Warning: No PubMed IDs found for the given query.")
      return self.pubmed_ids

    except requests.exceptions.RequestException as e:
      print(f"Error fetching PubMed Ids: {e}")
      return []

    except ET.ParseError as e:
      print("Error: Unable to parse PubMed response.")
      return []

  def retrieve_paper_details(self, pubmed_ids: List[str]) -> List[ET.Element]:
    all_papers = []
    if not self.pubmed_ids:
      print("Error: No PubMed IDs provided.")
      return []

    for i in range(0, len(self.pubmed_ids), self.batch_size):
      list_pubmed_ids = ','.join(self.pubmed_ids[i : i + self.batch_size])
      params = {
        'db': 'pubmed',
        'retmode': 'xml',
        'id': list_pubmed_ids
      }
      try:
        response = requests.get(EFETCH_URL, params=params)
        response.raise_for_status()
        parsed_xml_response = ET.fromstring(response.text)
        all_papers.append(parsed_xml_response)
      except requests.exceptions.RequestException as e:
        print(f"Error retrieving paper details: {e}")

      except ET.ParseError as e:
        print("Error: Unable to parse paper details response.")
    return all_papers

In [17]:
class PubMedParser():
  email_pattern = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
  academic_keywords = ['university', 'school', 'institute', 'department', 'college', 'lab', 'academy']
  academic_domains = ['edu', 'ac', '.org', '.gov']

  @staticmethod
  def _parse_paper_metadata(paper_elements: List[ET.Element]) -> List[List[str]]:
    parsed_metadata = []

    for root in paper_elements:
      pubmed_articles = root.findall('.//PubmedArticle')
      for article in pubmed_articles:
        try:
          pubmed_id = PubMedParser._extract_pubmed_id(article)
          article_title = PubMedParser._get_article_title(article)
          publication_date = PubMedParser._extract_publication_date(article)
          non_academic_authors, company_affiliations, corresponding_author_emails = PubMedParser._extract_authors(article)

          """ Including research papers with at least one non-academic author or papers with at least one author aliated with a pharmaceutical or biotech
              company
          """
          if len(non_academic_authors) > 0:
            non_academic_authors_str = ', '.join(non_academic_authors)
            company_affiliations_str = ', '.join(company_affiliations)
            corresponding_author_emails_str = list(filter(lambda x: x[1], corresponding_author_emails))
            paper_metadata = [pubmed_id, article_title, publication_date, non_academic_authors_str, company_affiliations_str, corresponding_author_emails_str]
            parsed_metadata.append(paper_metadata)

        except Exception as e:
          print(f"Error retrieving paper metadata: {e}")

    return parsed_metadata

  @staticmethod
  def _extract_pubmed_id(article: ET.Element) -> str:
    pubmed_id_elem = article.find('.//PMID')
    if pubmed_id_elem is not None:
      pubmed_id = pubmed_id_elem.text
      return pubmed_id

    return "PubMed ID Not Available"

  @staticmethod
  def _get_article_title(article: ET.Element) -> str:
    article_title_elem = article.find('.//ArticleTitle')
    if article_title_elem is not None:
        return article_title_elem.text

    return "Title Not Available"


  @staticmethod
  def _extract_publication_date(article: ET.Element) -> str:
    pub_date_elem = article.find('.//PubDate')
    year_elem = pub_date_elem.find('.//Year')
    if year_elem is not None:
      year = year_elem.text
      month_elem = pub_date_elem.find('.//Month')
      month = month_elem.text if month_elem is not None else '01' # Default to January if month is missing
      day_elem = pub_date_elem.find('.//Day')
      day = day_elem.text if day_elem is not None else '01' # Default to first day if day is missing
      pub_date = f"{year}-{month.zfill(2)}-{day.zfill(2)}"
      return pub_date
    else:
      return "Publication date not available"


In [18]:
def _extract_authors(article: ET.Element) -> Tuple[List[str], List[str], List[str]]:
  """
  Extracts author names, company affiliations, and corresponding author emails.

  Returns:
    - List of non-academic authors (company-affiliated).
    - List of company names associated with authors.
    - List of tuples (Author Name, Corresponding Email).
  """

  non_academic_authors = []
  company_affiliations = set()
  corresponding_author_emails = []
  author_elements = article.findall('.//Author')
  for author in author_elements:
    author_full_name = PubMedParser.retrieve_author_full_name(author)
    affiliation = PubMedParser._get_affiliation(author)
    corresponding_author_email = PubMedParser._retrieve_corresponding_author_email(author)

    if (PubMedParser._is_academic_affiliation(affiliation) or PubMedParser._is_academic_email(corresponding_author_email)):
      continue
    non_academic_authors.append(author_full_name)
    company_affiliations.add(affiliation)
    corresponding_author_emails.append((author_full_name, corresponding_author_email))

  return non_academic_authors, list(company_affiliations), corresponding_author_emails

PubMedParser._extract_authors = staticmethod(_extract_authors)

In [19]:
def retrieve_author_full_name(author: ET.Element) -> str:
  """Extracts and formats the author's full name."""
  first_name = author.find('.//ForeName')
  last_name = author.find('.//LastName')
  first_name = first_name.text if first_name is not None else ''
  last_name = last_name.text if last_name is not None else ''
  author_name = f"{first_name} {last_name}"
  return author_name

PubMedParser.retrieve_author_full_name = staticmethod(retrieve_author_full_name)

In [20]:
def _get_affiliation(author: ET.Element) -> str:
  """Extracts the author's affiliation if available."""
  affiliation_elem = author.find('.//Affiliation')
  affiliation = ""

  if affiliation_elem is not None:
    affiliation = affiliation_elem.text.strip()
    return affiliation
  return "Affiliation not available"

PubMedParser._get_affiliation = staticmethod(_get_affiliation)

In [21]:
def _is_academic_affiliation(affiliation: str) -> bool:
  """Checks if an affiliation is academic using heuristics email addresses, words like
      university, labs etc)"""
  if any(keyword in affiliation.lower() for keyword in PubMedParser.academic_keywords):
    return True
  return False

PubMedParser._is_academic_affiliation = _is_academic_affiliation

In [22]:
def _is_academic_email(email: str) -> bool:
  """Checks if the email domain contains any academic domain keywords."""
  if email:
    domain = email.split('@')[-1].lower()
    for academic_domain in PubMedParser.academic_domains:
      if academic_domain in domain:
        return True
  return False
PubMedParser._is_academic_email = _is_academic_email

In [23]:
def _retrieve_corresponding_author_email(author: ET.Element) -> str:
   """Extracts the author's email if available and non-academic."""
   affiliation_text = PubMedParser._get_affiliation(author).lower()
   email_match = re.search(PubMedParser.email_pattern, affiliation_text)

   email = None
   if email_match:
    email = email_match.group(0)
   return email

PubMedParser._retrieve_corresponding_author_email = staticmethod(_retrieve_corresponding_author_email)

In [24]:
class CSVWriter():
  @staticmethod
  def _save_to_csv(parsed_metadata: List[List[str]], filename: str = 'pubmed_results.csv'):
    headers = ["PubmedID", "Title", "Publication Date", "Non-academic Author(s)", "Company Affiliation(s)", "Corresponding Author Email(s)"]
    try:
      with open(filename, mode='w', newline="", encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        for row in parsed_metadata:
          row[5] =  "; ".join([f"{author_name} - {author_email}" for author_name, author_email in row[5]])
          writer.writerow(row)

        print(f"Results saved to {filename}")

    except Exception as e:
      print(f"Error saving results to CSV file: {e}")

In [25]:
if __name__ == "__main__":

    query = 'cancer AND brain'
    fetcher = PubMedFetcher(query)
    pubmed_ids = fetcher.fetch_pubmed_ids()
    paper_elements = fetcher.retrieve_paper_details(pubmed_ids)
    parsed_data = PubMedParser._parse_paper_metadata(paper_elements)
    CSVWriter._save_to_csv(parsed_data)


Results saved to pubmed_results.csv
